In [11]:
import numpy as np
import pandas as pd

from sklearn.metrics import mean_absolute_error, mean_squared_error


def calculate_mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def direction_accuracy(actual, forecast):
    actual_direction = np.sign(np.diff(actual))
    forecast_direction = np.sign(np.diff(forecast))
    correct = np.sum(actual_direction == forecast_direction)
    return correct / len(actual_direction) * 100

def rmse_reduction(rmse_model, rmse_rw):
    return (1 - rmse_model / rmse_rw) * 100


def calculate_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def calculate_mae(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)

def calculate_corr_dir(actual_values, predicted_values, last_value):
    if not last_value:
        return 1
    t = actual_values.tail(1).values[0]
    t2 = predicted_values.tail(1).values[0]
    sign_act = np.sign(actual_values.tail(1).values[0] - last_value)
    sign_pred = np.sign(predicted_values.tail(1).values[0] - last_value)
    if sign_act == sign_pred:
        return 1
    else:
        return 0

def calculate_avg_dir_accuracy(values):
    return sum(values) / len(values) * 100


def calculate_rmse_reduction(baseline_rmse, model_rmse):
    return (baseline_rmse - model_rmse) / baseline_rmse * 100





segments = ["csz", "pmx", "smx"]
horizons = [1, 2, 5, 10, 20]

segments = ["pmx"]
horizons = [20]

for seg in segments:
    for hor in horizons:
        df_ml_0 = pd.read_csv(f"./pred/{seg}_{hor}_0.csv")
        df_ml_1 = pd.read_csv(f"./pred/{seg}_{hor}_1.csv")
        df_ml_2 = pd.read_csv(f"./pred/{seg}_{hor}_2.csv")

        df_r = pd.read_csv(f"./pred/{seg}_{hor}_r.csv")

        df_r.rename(columns={"Actuals": "Actual"}, inplace=True)
        df_r.columns = df_r.columns.str.replace('_fcs', '_pred', regex=False)


        #df_r = df_r.iloc[1:-27]
        df_r.head()
        '''
        # Define a small delta
        delta = 2e-3
        ml_long = False
        # Find the longer DataFrame
        if len(df_ml_0) > len(df_r):
            longer_df = df_ml_0
            shorter_df = df_r
            ml_long = True
        else:
            longer_df = df_r
            shorter_df = df_ml_0

        # Initialize counters
        i = 0
        j = 0

        # Compare each row of both DataFrames
        while i < len(longer_df) and j < len(shorter_df):
            if not np.isclose(longer_df.loc[i, 'Actual'], shorter_df.loc[j, 'Actual'], atol=delta):
                longer_df = longer_df.drop(i).reset_index(drop=True)
                if ml_long:
                    df_ml_1_df = df_ml_1.drop(i).reset_index(drop=True)
                    df_ml_2_df = df_ml_2.drop(i).reset_index(drop=True)
            else:
                i += 2
                j += 2

        # Remove any extra rows from the longer DataFrame
        longer_df = longer_df[:len(shorter_df)]
        if ml_long:
            df_ml_1 = df_ml_1[:len(shorter_df)]
            df_ml_2 = df_ml_2[:len(shorter_df)]



        # Print the final cleaned longer DataFrame
        print("\nCleaned Longer DataFrame:")
        print(longer_df)
        '''
        cumsum = 0
        for i in range(len(df_r)):
            t = df_r.loc[i, "Actual"]
            cumsum += df_r.loc[i, "Actual"] - df_ml_0.loc[i, "Actual"]
            
        
        print(cumsum)

        # Sum up all column errors to get the total cumulative error

        # Remove the identified rows from the longer DataFrame
        df_merged = pd.merge(df_r, df_ml_0, left_index=True, right_index=True, how="left", suffixes=("", "_ml"))
        df_merged = pd.merge(df_merged, df_ml_1, left_index=True, right_index=True, how="left", suffixes=("", "_1_ml"))
        df_merged = pd.merge(df_merged, df_ml_2, left_index=True, right_index=True, how="left", suffixes=("", "_2_ml"))


        # Remove right DataFrame's duplicate columns
        columns_to_keep = [col for col in df_merged.columns if not col.endswith('_ml')]

        # Keep only the required columns (excluding duplicates from the right DataFrame)
        df_final = df_merged[columns_to_keep]
        # Remove unnamed columns (columns with 'Unnamed' in their name)
        df_final = df_final.loc[:, ~df_final.columns.str.contains('^Unnamed')]
        #df_final.head()
        #print(df_final)
        
        
        df = df_final
        
        # COMB 1
        # Selected models
        if seg == "pmx" or seg == "csz":
            forecast_columns = ['VECM_pred', 'LSTM_pred_[2, 4]']
        elif seg == "smx":
            forecast_columns = ['VECM_pred', 'LSTM_pred_[3, 4]']
            
        forecast_data = df[forecast_columns].to_numpy()


        df['avg_1_pred'] = df[forecast_columns].mean(axis=1)
        df['avg_1_res'] = df['Actual'] - df['avg_1_pred']


        # Covariance matrix and eigenvectors
        cov_matrix = np.cov(forecast_data, rowvar=False)
        eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)

        # Get the eigenvector associated with the largest eigenvalue
        largest_eigenvector = eigenvectors[:, np.argmax(eigenvalues)]
        eigenvector_weights = largest_eigenvector / np.sum(largest_eigenvector)

        # Combine forecasts using the largest eigenvector
        df['eigen_1_pred'] = np.dot(forecast_data, eigenvector_weights)
        df['eigen_1_res'] = df['Actual'] - df['eigen_1_pred']
        #print(df)


        # COMB 2
        # ALl models
        # Prepare for Eigenvector-Based Combination
        if seg == "pmx":
            forecast_columns = ['VAR_pred', 'VECM_pred', 'ARIMA_pred', 'MLP_pred_[]', 'LSTM_pred_[]', 
                                'MLP_pred_[2]', 'LSTM_pred_[2]', 'MLP_pred_[2, 4]', 'LSTM_pred_[2, 4]']
        elif seg == "csz":
            forecast_columns = ['VAR_pred', 'VECM_pred', 'ARIMA_pred', 'MLP_pred_[]', 'LSTM_pred_[]', 
                                'MLP_pred_[4]', 'LSTM_pred_[4]', 'MLP_pred_[2, 4]', 'LSTM_pred_[2, 4]']     
        elif seg == "smx":
            forecast_columns = ['VAR_pred', 'VECM_pred', 'ARIMA_pred', 'MLP_pred_[]', 'LSTM_pred_[]', 
                                'MLP_pred_[2]', 'LSTM_pred_[2]', 'MLP_pred_[3, 4]', 'LSTM_pred_[3, 4]']       
        forecast_data = df[forecast_columns].to_numpy()



        # Compute the Simple Average of All Models
        df['avg_2_pred'] = df[forecast_columns].mean(axis=1)
        df['avg_2_res'] = df['Actual'] - df['avg_2_pred']


        # Combine All Models using the Eigenvector-Based Combination
        cov_matrix_all = np.cov(forecast_data, rowvar=False)
        eigenvalues_all, eigenvectors_all = np.linalg.eig(cov_matrix_all)

        # Get the eigenvector associated with the largest eigenvalue
        largest_eigenvector_all = eigenvectors_all[:, np.argmax(eigenvalues_all)]
        eigenvector_weights_all = largest_eigenvector_all / np.sum(largest_eigenvector_all)

        # Combine forecasts using the largest eigenvector
        df['eigen_2_pred'] = np.dot(forecast_data, eigenvector_weights_all)
        df['eigen_2_res'] = df['Actual'] - df['eigen_2_pred']

        #print(df)

        
        

        
        
        # COMB 3: ALl Econometric models

        #df = df_final.copy()
        #print(df)

        # Prepare for Eigenvector-Based Combination
        forecast_columns = ['VAR_pred', 'VECM_pred', 'ARIMA_pred']
        forecast_data = df[forecast_columns].to_numpy()

        # Compute the Simple Average of All Models
        df['avg_3_pred'] = df[forecast_columns].mean(axis=1)
        df['avg_3_res'] = df['Actual'] - df['avg_3_pred']

        # Combine All Models using the Eigenvector-Based Combination
        cov_matrix_all = np.cov(forecast_data, rowvar=False)
        eigenvalues_all, eigenvectors_all = np.linalg.eig(cov_matrix_all)

        # Get the eigenvector associated with the largest eigenvalue
        largest_eigenvector_all = eigenvectors_all[:, np.argmax(eigenvalues_all)]
        eigenvector_weights_all = largest_eigenvector_all / np.sum(largest_eigenvector_all)

        # Combine forecasts using the largest eigenvector
        df['eigen_3_pred'] = np.dot(forecast_data, eigenvector_weights_all)
        df['eigen_3_res'] = df['Actual'] - df['eigen_3_pred']

        df_comb2 = df.copy()
        #print(df)
        
        
        # COMB 4: ALl ML models

        #df = df_final.copy()
        #print(df)


        # Prepare for Eigenvector-Based Combination
        if seg == "pmx":
            forecast_columns = ['MLP_pred_[]', 'LSTM_pred_[]', 
                                'MLP_pred_[2]', 'LSTM_pred_[2]', 'MLP_pred_[2, 4]', 'LSTM_pred_[2, 4]']
        elif seg == "csz":
            forecast_columns = ['MLP_pred_[]', 'LSTM_pred_[]', 
                                'MLP_pred_[4]', 'LSTM_pred_[4]', 'MLP_pred_[2, 4]', 'LSTM_pred_[2, 4]']     
        elif seg == "smx":
            forecast_columns = ['MLP_pred_[]', 'LSTM_pred_[]', 
                                'MLP_pred_[2]', 'LSTM_pred_[2]', 'MLP_pred_[3, 4]', 'LSTM_pred_[3, 4]']  
        forecast_data = df[forecast_columns].to_numpy()



        # Compute the Simple Average of All Models
        df['avg_4_pred'] = df[forecast_columns].mean(axis=1)
        df['avg_4_res'] = df['Actual'] - df['avg_4_pred']


        # Combine All Models using the Eigenvector-Based Combination
        cov_matrix_all = np.cov(forecast_data, rowvar=False)
        eigenvalues_all, eigenvectors_all = np.linalg.eig(cov_matrix_all)

        # Get the eigenvector associated with the largest eigenvalue
        largest_eigenvector_all = eigenvectors_all[:, np.argmax(eigenvalues_all)]
        eigenvector_weights_all = largest_eigenvector_all / np.sum(largest_eigenvector_all)

        # Combine forecasts using the largest eigenvector
        df['eigen_4_pred'] = np.dot(forecast_data, eigenvector_weights_all)
        df['eigen_4_res'] = df['Actual'] - df['eigen_4_pred']

        df_comb2 = df.copy()
        #print(df)
        

        df.to_csv(f'./mcs/{seg}_{hor}.csv', index=False, mode="a")    



        rw_rmses= []

        avg_1_rmses = []
        avg_1_maes = []
        avg_1_mapes = []
        avg_1_corr_dirs = []
        eigen_1_rmses = []
        eigen_1_maes = []
        eigen_1_mapes = []
        eigen_1_corr_dirs = []

        avg_2_rmses = []
        avg_2_maes = []
        avg_2_mapes = []
        avg_2_corr_dirs = []
        eigen_2_rmses = []
        eigen_2_maes = []
        eigen_2_mapes = []
        eigen_2_corr_dirs = []

        avg_3_rmses = []
        avg_3_maes = []
        avg_3_mapes = []
        avg_3_corr_dirs = []
        eigen_3_rmses = []
        eigen_3_maes = []
        eigen_3_mapes = []
        eigen_3_corr_dirs = []

        avg_4_rmses = []
        avg_4_maes = []
        avg_4_mapes = []
        avg_4_corr_dirs = []
        eigen_4_rmses = []
        eigen_4_maes = []
        eigen_4_mapes = []
        eigen_4_corr_dirs = []

        num_fcs = len(df) // hor
        for j in range(num_fcs):
            i = j * hor
            if j == 0:
                last_value = None
            else:
                last_value = df["Actual"].iloc[i-1]
            act = df["Actual"].iloc[i:i+hor]
            print(act)
            
            rw_rmses.append(calculate_rmse(act, df["RW_pred"].iloc[i:i+hor]))
            
            pred = df["avg_1_pred"].iloc[i:i+hor]
            avg_1_rmses.append(calculate_rmse(act,pred ))
            avg_1_maes.append(calculate_mae(act, pred))
            avg_1_mapes.append(calculate_mape(act, pred))
            if last_value:
                avg_1_corr_dirs.append(calculate_corr_dir(act, pred, last_value))
            
            pred = df["eigen_1_pred"].iloc[i:i+hor]   
            eigen_1_rmses.append(calculate_rmse(act,pred ))
            eigen_1_maes.append(calculate_mae(act, pred))
            eigen_1_mapes.append(calculate_mape(act, pred))
            if last_value:
                eigen_1_corr_dirs.append(calculate_corr_dir(act, pred, last_value))

            pred = df["avg_2_pred"].iloc[i:i+hor]
            avg_2_rmses.append(calculate_rmse(act,pred ))
            avg_2_maes.append(calculate_mae(act, pred))
            avg_2_mapes.append(calculate_mape(act, pred))
            if last_value:
                avg_2_corr_dirs.append(calculate_corr_dir(act, pred, last_value))
            
            pred = df["eigen_2_pred"].iloc[i:i+hor]   
            eigen_2_rmses.append(calculate_rmse(act,pred ))
            eigen_2_maes.append(calculate_mae(act, pred))
            eigen_2_mapes.append(calculate_mape(act, pred))
            if last_value:
                eigen_2_corr_dirs.append(calculate_corr_dir(act, pred, last_value))

            pred = df["avg_3_pred"].iloc[i:i+hor]
            avg_3_rmses.append(calculate_rmse(act,pred ))
            avg_3_maes.append(calculate_mae(act, pred))
            avg_3_mapes.append(calculate_mape(act, pred))
            if last_value:
                avg_3_corr_dirs.append(calculate_corr_dir(act, pred, last_value))
            
            pred = df["eigen_3_pred"].iloc[i:i+hor]   
            eigen_3_rmses.append(calculate_rmse(act,pred ))
            eigen_3_maes.append(calculate_mae(act, pred))
            eigen_3_mapes.append(calculate_mape(act, pred))
            if last_value:
                eigen_3_corr_dirs.append(calculate_corr_dir(act, pred, last_value))

            pred = df["avg_4_pred"].iloc[i:i+hor]
            avg_4_rmses.append(calculate_rmse(act,pred ))
            avg_4_maes.append(calculate_mae(act, pred))
            avg_4_mapes.append(calculate_mape(act, pred))
            if last_value:
                avg_4_corr_dirs.append(calculate_corr_dir(act, pred, last_value))
            
            pred = df["eigen_4_pred"].iloc[i:i+hor]   
            eigen_4_rmses.append(calculate_rmse(act,pred ))
            eigen_4_maes.append(calculate_mae(act, pred))
            eigen_4_mapes.append(calculate_mape(act, pred))
            if last_value:
                eigen_4_corr_dirs.append(calculate_corr_dir(act, pred, last_value))
            

        rw_rmse = np.mean(rw_rmses)

        avg_1_rmse = np.mean(avg_1_rmses)
        avg_1_mae = np.mean(avg_1_maes)
        avg_1_mape = np.mean(avg_1_mapes)
        avg_1_corr_dir = calculate_avg_dir_accuracy(avg_1_corr_dirs)
        avg_1_red = calculate_rmse_reduction(rw_rmse, avg_1_rmse)
                
        eigen_1_rmse = np.mean(eigen_1_rmses)
        eigen_1_mae = np.mean(eigen_1_maes)
        eigen_1_mape = np.mean(eigen_1_mapes)
        eigen_1_corr_dir = calculate_avg_dir_accuracy(eigen_1_corr_dirs)
        eigen_1_red = calculate_rmse_reduction(rw_rmse, eigen_1_rmse)

        avg_2_rmse = np.mean(avg_2_rmses)
        avg_2_mae = np.mean(avg_2_maes)
        avg_2_mape = np.mean(avg_2_mapes)
        avg_2_corr_dir = calculate_avg_dir_accuracy(avg_2_corr_dirs)
        avg_2_red = calculate_rmse_reduction(rw_rmse, avg_2_rmse)

        eigen_2_rmse = np.mean(eigen_2_rmses)
        eigen_2_mae = np.mean(eigen_2_maes)
        eigen_2_mape = np.mean(eigen_2_mapes)
        eigen_2_corr_dir = calculate_avg_dir_accuracy(eigen_2_corr_dirs)
        eigen_2_red = calculate_rmse_reduction(rw_rmse, eigen_2_rmse)

        avg_3_rmse = np.mean(avg_3_rmses)
        avg_3_mae = np.mean(avg_3_maes)
        avg_3_mape = np.mean(avg_3_mapes)
        avg_3_corr_dir = calculate_avg_dir_accuracy(avg_3_corr_dirs)
        avg_3_red = calculate_rmse_reduction(rw_rmse, avg_3_rmse)

        eigen_3_rmse = np.mean(eigen_3_rmses)
        eigen_3_mae = np.mean(eigen_3_maes)
        eigen_3_mape = np.mean(eigen_3_mapes)
        eigen_3_corr_dir = calculate_avg_dir_accuracy(eigen_3_corr_dirs)
        eigen_3_red = calculate_rmse_reduction(rw_rmse, eigen_3_rmse)

        avg_4_rmse = np.mean(avg_4_rmses)
        avg_4_mae = np.mean(avg_4_maes)
        avg_4_mape = np.mean(avg_4_mapes)
        avg_4_corr_dir = calculate_avg_dir_accuracy(avg_4_corr_dirs)
        avg_4_red = calculate_rmse_reduction(rw_rmse, avg_4_rmse)

        eigen_4_rmse = np.mean(eigen_4_rmses)
        eigen_4_mae = np.mean(eigen_4_maes)
        eigen_4_mape = np.mean(eigen_4_mapes)
        eigen_4_corr_dir = calculate_avg_dir_accuracy(eigen_4_corr_dirs)
        eigen_4_red = calculate_rmse_reduction(rw_rmse, eigen_4_rmse)

        # Prepare data for CSV
        results = [
            [seg, hor, 'avg_1', avg_1_rmse, avg_1_mae, avg_1_mape, avg_1_corr_dir, avg_1_red],
            [seg, hor, 'eigen_1', eigen_1_rmse, eigen_1_mae, eigen_1_mape, eigen_1_corr_dir, eigen_1_red],
            [seg, hor, 'avg_2', avg_2_rmse, avg_2_mae, avg_2_mape, avg_2_corr_dir, avg_2_red],
            [seg, hor, 'eigen_2', eigen_2_rmse, eigen_2_mae, eigen_2_mape, eigen_2_corr_dir, eigen_2_red],
            [seg, hor, 'avg_3', avg_3_rmse, avg_3_mae, avg_3_mape, avg_3_corr_dir, avg_3_red],
            [seg, hor, 'eigen_3', eigen_3_rmse, eigen_3_mae, eigen_3_mape, eigen_3_corr_dir, eigen_3_red],
            [seg, hor, 'avg_4', avg_4_rmse, avg_4_mae, avg_4_mape, avg_4_corr_dir, avg_4_red],
            [seg, hor, 'eigen_4', eigen_4_rmse, eigen_4_mae, eigen_4_mape, eigen_4_corr_dir, eigen_4_red],
        ]

        # Create a DataFrame
        columns = ['Segment', 'Horizon', 'Model', 'RMSE', 'MAE', 'MAPE', 'CorrDir', 'Reduction']
        results_df = pd.DataFrame(results, columns=columns)

        # Write to CSV
        results_df.to_csv('model_metrics.csv', index=False, header=False, mode="a")    

9.823253321883385e-13
0     10.143960
1     10.172904
2     10.146826
3     10.101641
4     10.092743
5     10.090423
6     10.084725
7     10.050657
8      9.992048
9      9.953134
10     9.930324
11     9.897570
12     9.804109
13     9.722565
14     9.681344
15     9.664405
16     9.730383
17     9.840228
18     9.892578
19     9.934599
Name: Actual, dtype: float64
20     9.983869
21    10.037581
22    10.036269
23    10.013731
24     9.995839
25     9.981050
26     9.995337
27    10.019001
28    10.030517
29    10.051002
30    10.098149
31    10.120452
32    10.137017
33    10.157742
34    10.181271
35    10.176183
36    10.158827
37    10.139192
38    10.120331
39    10.100123
Name: Actual, dtype: float64
40    10.101231
41    10.106347
42    10.100575
43    10.095017
44    10.083682
45    10.067603
46    10.055436
47    10.064968
48    10.080629
49    10.100575
50    10.115853
51    10.129188
52    10.136661
53    10.150504
54    10.175269
55    10.221941
56    10.253510
57    10

In [12]:

import numpy as np
import pandas as pd

from sklearn.metrics import mean_absolute_error, mean_squared_error


def calculate_mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def direction_accuracy(actual, forecast):
    actual_direction = np.sign(np.diff(actual))
    forecast_direction = np.sign(np.diff(forecast))
    correct = np.sum(actual_direction == forecast_direction)
    return correct / len(actual_direction) * 100

def rmse_reduction(rmse_model, rmse_rw):
    return (1 - rmse_model / rmse_rw) * 100


def calculate_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def calculate_mae(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)

def calculate_corr_dir(actual_values, predicted_values, last_value):
    if not last_value:
        return 1
    t = actual_values.tail(1).values[0]
    t2 = predicted_values.tail(1).values[0]
    sign_act = np.sign(actual_values.tail(1).values[0] - last_value)
    sign_pred = np.sign(predicted_values.tail(1).values[0] - last_value)
    if sign_act == sign_pred:
        return 1
    else:
        return 0

def calculate_avg_dir_accuracy(values):
    return sum(values) / len(values) * 100


def calculate_rmse_reduction(baseline_rmse, model_rmse):
    return (baseline_rmse - model_rmse) / baseline_rmse * 100





segments = ["pmx"]
horizons = [1,2,5,20]

#segments = ["pmx", "smx"]
#horizons = [1, 2, 5, 10, 20]

for seg in segments:
    for hor in horizons:
        df_ml_0 = pd.read_csv(f"./pred/{seg}_{hor}_0.csv")
        df_ml_1 = pd.read_csv(f"./pred/{seg}_{hor}_1.csv")
        df_ml_2 = pd.read_csv(f"./pred/{seg}_{hor}_2.csv")

        df_r = pd.read_csv(f"./pred/{seg}_{hor}_r.csv")

        df_r.rename(columns={"Actuals": "Actual"}, inplace=True)
        df_r.columns = df_r.columns.str.replace('_fcs', '_pred', regex=False)


        #df_r = df_r.iloc[1:-27]
        df_r.head()
        '''
        # Define a small delta
        delta = 2e-3
        ml_long = False
        # Find the longer DataFrame
        if len(df_ml_0) > len(df_r):
            longer_df = df_ml_0
            shorter_df = df_r
            ml_long = True
        else:
            longer_df = df_r
            shorter_df = df_ml_0

        # Initialize counters
        i = 0
        j = 0

        # Compare each row of both DataFrames
        while i < len(longer_df) and j < len(shorter_df):
            if not np.isclose(longer_df.loc[i, 'Actual'], shorter_df.loc[j, 'Actual'], atol=delta):
                longer_df = longer_df.drop(i).reset_index(drop=True)
                if ml_long:
                    df_ml_1_df = df_ml_1.drop(i).reset_index(drop=True)
                    df_ml_2_df = df_ml_2.drop(i).reset_index(drop=True)
            else:
                i += 2
                j += 2

        # Remove any extra rows from the longer DataFrame
        longer_df = longer_df[:len(shorter_df)]
        if ml_long:
            df_ml_1 = df_ml_1[:len(shorter_df)]
            df_ml_2 = df_ml_2[:len(shorter_df)]



        # Print the final cleaned longer DataFrame
        print("\nCleaned Longer DataFrame:")
        print(longer_df)
        '''
        cumsum = 0
        for i in range(len(df_r)):
            t = df_r.loc[i, "Actual"]
            cumsum += df_r.loc[i, "Actual"] - df_ml_0.loc[i, "Actual"]
            
        
        print(cumsum)

        # Sum up all column errors to get the total cumulative error

        # Remove the identified rows from the longer DataFrame
        df_merged = pd.merge(df_r, df_ml_0, left_index=True, right_index=True, how="left", suffixes=("", "_ml"))
        df_merged = pd.merge(df_merged, df_ml_1, left_index=True, right_index=True, how="left", suffixes=("", "_1_ml"))
        df_merged = pd.merge(df_merged, df_ml_2, left_index=True, right_index=True, how="left", suffixes=("", "_2_ml"))


        # Remove right DataFrame's duplicate columns
        columns_to_keep = [col for col in df_merged.columns if not col.endswith('_ml')]

        # Keep only the required columns (excluding duplicates from the right DataFrame)
        df_final = df_merged[columns_to_keep]
        # Remove unnamed columns (columns with 'Unnamed' in their name)
        df_final = df_final.loc[:, ~df_final.columns.str.contains('^Unnamed')]
        #df_final.head()
        #print(df_final)
        
        
        df = df_final
        
        # COMB 1
        # Selected models
        if seg == "pmx" or seg == "csz":
            forecast_columns = ['VECM_pred', 'LSTM_pred_[2, 4]']
        elif seg == "smx":
            forecast_columns = ['VECM_pred', 'LSTM_pred_[3, 4]']
            
        forecast_data = df[forecast_columns].to_numpy()


        df['avg_1_pred'] = df[forecast_columns].mean(axis=1)
        df['avg_1_res'] = df['Actual'] - df['avg_1_pred']


        # Covariance matrix and eigenvectors
        cov_matrix = np.cov(forecast_data, rowvar=False)
        eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)

        # Get the eigenvector associated with the largest eigenvalue
        largest_eigenvector = eigenvectors[:, np.argmax(eigenvalues)]
        eigenvector_weights = largest_eigenvector / np.sum(largest_eigenvector)

        # Combine forecasts using the largest eigenvector
        df['eigen_1_pred'] = np.dot(forecast_data, eigenvector_weights)
        df['eigen_1_res'] = df['Actual'] - df['eigen_1_pred']
        #print(df)


        # COMB 2
        # ALl models
        # Prepare for Eigenvector-Based Combination
        if seg == "pmx":
            forecast_columns = ['VAR_pred', 'VECM_pred', 'ARIMA_pred', 'MLP_pred_[]', 'LSTM_pred_[]', 
                                'MLP_pred_[2]', 'LSTM_pred_[2]', 'MLP_pred_[2, 4]', 'LSTM_pred_[2, 4]']
        elif seg == "csz":
            forecast_columns = ['VAR_pred', 'VECM_pred', 'ARIMA_pred', 'MLP_pred_[]', 'LSTM_pred_[]', 
                                'MLP_pred_[4]', 'LSTM_pred_[4]', 'MLP_pred_[2, 4]', 'LSTM_pred_[2, 4]']     
        elif seg == "smx":
            forecast_columns = ['VAR_pred', 'VECM_pred', 'ARIMA_pred', 'MLP_pred_[]', 'LSTM_pred_[]', 
                                'MLP_pred_[2]', 'LSTM_pred_[2]', 'MLP_pred_[3, 4]', 'LSTM_pred_[3, 4]']       
        forecast_data = df[forecast_columns].to_numpy()



        # Compute the Simple Average of All Models
        df['avg_2_pred'] = df[forecast_columns].mean(axis=1)
        df['avg_2_res'] = df['Actual'] - df['avg_2_pred']


        # Combine All Models using the Eigenvector-Based Combination
        cov_matrix_all = np.cov(forecast_data, rowvar=False)
        eigenvalues_all, eigenvectors_all = np.linalg.eig(cov_matrix_all)

        # Get the eigenvector associated with the largest eigenvalue
        largest_eigenvector_all = eigenvectors_all[:, np.argmax(eigenvalues_all)]
        eigenvector_weights_all = largest_eigenvector_all / np.sum(largest_eigenvector_all)

        # Combine forecasts using the largest eigenvector
        df['eigen_2_pred'] = np.dot(forecast_data, eigenvector_weights_all)
        df['eigen_2_res'] = df['Actual'] - df['eigen_2_pred']

        #print(df)

        
        

        
        
        # COMB 3: ALl Econometric models

        #df = df_final.copy()
        #print(df)

        # Prepare for Eigenvector-Based Combination
        forecast_columns = ['VAR_pred', 'VECM_pred', 'ARIMA_pred']
        forecast_data = df[forecast_columns].to_numpy()

        # Compute the Simple Average of All Models
        df['avg_3_pred'] = df[forecast_columns].mean(axis=1)
        df['avg_3_res'] = df['Actual'] - df['avg_3_pred']

        # Combine All Models using the Eigenvector-Based Combination
        cov_matrix_all = np.cov(forecast_data, rowvar=False)
        eigenvalues_all, eigenvectors_all = np.linalg.eig(cov_matrix_all)

        # Get the eigenvector associated with the largest eigenvalue
        largest_eigenvector_all = eigenvectors_all[:, np.argmax(eigenvalues_all)]
        eigenvector_weights_all = largest_eigenvector_all / np.sum(largest_eigenvector_all)

        # Combine forecasts using the largest eigenvector
        df['eigen_3_pred'] = np.dot(forecast_data, eigenvector_weights_all)
        df['eigen_3_res'] = df['Actual'] - df['eigen_3_pred']

        df_comb2 = df.copy()
        #print(df)
        
        
        # COMB 4: ALl ML models

        #df = df_final.copy()
        #print(df)


        # Prepare for Eigenvector-Based Combination
        if seg == "pmx":
            forecast_columns = ['MLP_pred_[]', 'LSTM_pred_[]', 
                                'MLP_pred_[2]', 'LSTM_pred_[2]', 'MLP_pred_[2, 4]', 'LSTM_pred_[2, 4]']
            col1_mlp = 'MLP_pred_[2]'
            col1_lstm = 'LSTM_pred_[2]'
            col2_mlp = 'MLP_pred_[2, 4]'
            col2_lstm = 'LSTM_pred_[2, 4]'
        elif seg == "csz":
            forecast_columns = ['MLP_pred_[]', 'LSTM_pred_[]', 
                                'MLP_pred_[4]', 'LSTM_pred_[4]', 'MLP_pred_[2, 4]', 'LSTM_pred_[2, 4]']    
            col1_mlp = 'MLP_pred_[4]'
            col1_lstm = 'LSTM_pred_[4]'
            col2_mlp = 'MLP_pred_[2, 4]'
            col2_lstm = 'LSTM_pred_[2, 4]'
        elif seg == "smx":
            forecast_columns = ['MLP_pred_[]', 'LSTM_pred_[]', 
                                'MLP_pred_[2]', 'LSTM_pred_[2]', 'MLP_pred_[3, 4]', 'LSTM_pred_[3, 4]']  
            col1_mlp = 'MLP_pred_[2]'
            col1_lstm = 'LSTM_pred_[2]'
            col2_mlp = 'MLP_pred_[3, 4]'
            col2_lstm = 'LSTM_pred_[3, 4]'
        forecast_data = df[forecast_columns].to_numpy()



        # Compute the Simple Average of All Models
        df['avg_4_pred'] = df[forecast_columns].mean(axis=1)
        df['avg_4_res'] = df['Actual'] - df['avg_4_pred']


        # Combine All Models using the Eigenvector-Based Combination
        cov_matrix_all = np.cov(forecast_data, rowvar=False)
        eigenvalues_all, eigenvectors_all = np.linalg.eig(cov_matrix_all)

        # Get the eigenvector associated with the largest eigenvalue
        largest_eigenvector_all = eigenvectors_all[:, np.argmax(eigenvalues_all)]
        eigenvector_weights_all = largest_eigenvector_all / np.sum(largest_eigenvector_all)

        # Combine forecasts using the largest eigenvector
        df['eigen_4_pred'] = np.dot(forecast_data, eigenvector_weights_all)
        df['eigen_4_res'] = df['Actual'] - df['eigen_4_pred']

        df_comb2 = df.copy()
        #print(df)
        

        df.to_csv(f'./mcs/{seg}_{hor}.csv', index=False, mode="a")    



        rw_rmses= []
        
        
        mlp_rmses = []
        mlp_maes = []
        mlp_mapes = []
        mlp_corr_dirs = []

        lstm_rmses = []
        lstm_maes = []
        lstm_mapes = []
        lstm_corr_dirs = []

        mlp_1_rmses = []
        mlp_1_maes = []
        mlp_1_mapes = []
        mlp_1_corr_dirs = []

        lstm_1_rmses = []
        lstm_1_maes = []
        lstm_1_mapes = []
        lstm_1_corr_dirs = []
        
        mlp_2_rmses = []
        mlp_2_maes = []
        mlp_2_mapes = []
        mlp_2_corr_dirs = []

        lstm_2_rmses = []
        lstm_2_maes = []
        lstm_2_mapes = []
        lstm_2_corr_dirs = []

        var_rmses = []
        var_maes = []
        var_mapes = []
        var_corr_dirs = []

        vecm_rmses = []
        vecm_maes = []
        vecm_mapes = []
        vecm_corr_dirs = []

        arima_rmses = []
        arima_maes = []
        arima_mapes = []
        arima_corr_dirs = []

        avg_1_rmses = []
        avg_1_maes = []
        avg_1_mapes = []
        avg_1_corr_dirs = []
        eigen_1_rmses = []
        eigen_1_maes = []
        eigen_1_mapes = []
        eigen_1_corr_dirs = []

        avg_2_rmses = []
        avg_2_maes = []
        avg_2_mapes = []
        avg_2_corr_dirs = []
        eigen_2_rmses = []
        eigen_2_maes = []
        eigen_2_mapes = []
        eigen_2_corr_dirs = []

        avg_3_rmses = []
        avg_3_maes = []
        avg_3_mapes = []
        avg_3_corr_dirs = []
        eigen_3_rmses = []
        eigen_3_maes = []
        eigen_3_mapes = []
        eigen_3_corr_dirs = []

        avg_4_rmses = []
        avg_4_maes = []
        avg_4_mapes = []
        avg_4_corr_dirs = []
        eigen_4_rmses = []
        eigen_4_maes = []
        eigen_4_mapes = []
        eigen_4_corr_dirs = []

        num_fcs = len(df) // hor
        for j in range(num_fcs):
            i = j * hor
            if j == 0:
                last_value = None
            else:
                last_value = df["Actual"].iloc[i-1]
            act = df["Actual"].iloc[i:i+hor]
            
            rw_rmses.append(calculate_rmse(act, df["RW_pred"].iloc[i:i+hor]))
            
            pred = df["avg_1_pred"].iloc[i:i+hor]
            avg_1_rmses.append(calculate_rmse(act,pred ))
            avg_1_maes.append(calculate_mae(act, pred))
            avg_1_mapes.append(calculate_mape(act, pred))
            if last_value:
                avg_1_corr_dirs.append(calculate_corr_dir(act, pred, last_value))
            
            pred = df["eigen_1_pred"].iloc[i:i+hor]   
            eigen_1_rmses.append(calculate_rmse(act,pred ))
            eigen_1_maes.append(calculate_mae(act, pred))
            eigen_1_mapes.append(calculate_mape(act, pred))
            if last_value:
                eigen_1_corr_dirs.append(calculate_corr_dir(act, pred, last_value))

            pred = df["avg_2_pred"].iloc[i:i+hor]
            avg_2_rmses.append(calculate_rmse(act,pred ))
            avg_2_maes.append(calculate_mae(act, pred))
            avg_2_mapes.append(calculate_mape(act, pred))
            if last_value:
                avg_2_corr_dirs.append(calculate_corr_dir(act, pred, last_value))
            
            pred = df["eigen_2_pred"].iloc[i:i+hor]   
            eigen_2_rmses.append(calculate_rmse(act,pred ))
            eigen_2_maes.append(calculate_mae(act, pred))
            eigen_2_mapes.append(calculate_mape(act, pred))
            if last_value:
                eigen_2_corr_dirs.append(calculate_corr_dir(act, pred, last_value))

            pred = df["avg_3_pred"].iloc[i:i+hor]
            avg_3_rmses.append(calculate_rmse(act,pred ))
            avg_3_maes.append(calculate_mae(act, pred))
            avg_3_mapes.append(calculate_mape(act, pred))
            if last_value:
                avg_3_corr_dirs.append(calculate_corr_dir(act, pred, last_value))
            
            pred = df["eigen_3_pred"].iloc[i:i+hor]   
            eigen_3_rmses.append(calculate_rmse(act,pred ))
            eigen_3_maes.append(calculate_mae(act, pred))
            eigen_3_mapes.append(calculate_mape(act, pred))
            if last_value:
                eigen_3_corr_dirs.append(calculate_corr_dir(act, pred, last_value))

            pred = df["avg_4_pred"].iloc[i:i+hor]
            avg_4_rmses.append(calculate_rmse(act,pred ))
            avg_4_maes.append(calculate_mae(act, pred))
            avg_4_mapes.append(calculate_mape(act, pred))
            if last_value:
                avg_4_corr_dirs.append(calculate_corr_dir(act, pred, last_value))
            
            pred = df["eigen_4_pred"].iloc[i:i+hor]   
            eigen_4_rmses.append(calculate_rmse(act,pred ))
            eigen_4_maes.append(calculate_mae(act, pred))
            eigen_4_mapes.append(calculate_mape(act, pred))
            if last_value:
                eigen_4_corr_dirs.append(calculate_corr_dir(act, pred, last_value))
                
                

            pred = df["MLP_pred_[]"].iloc[i:i + hor]
            mlp_rmses.append(calculate_rmse(act, pred))
            mlp_maes.append(calculate_mae(act, pred))
            mlp_mapes.append(calculate_mape(act, pred))
            if last_value:
                mlp_corr_dirs.append(calculate_corr_dir(act, pred, last_value))

            pred = df["LSTM_pred_[]"].iloc[i:i + hor]
            lstm_rmses.append(calculate_rmse(act, pred))
            lstm_maes.append(calculate_mae(act, pred))
            lstm_mapes.append(calculate_mape(act, pred))
            if last_value:
                lstm_corr_dirs.append(calculate_corr_dir(act, pred, last_value))
                
            pred = df[col1_mlp].iloc[i:i + hor]
            mlp_1_rmses.append(calculate_rmse(act, pred))
            mlp_1_maes.append(calculate_mae(act, pred))
            mlp_1_mapes.append(calculate_mape(act, pred))
            if last_value:
                mlp_1_corr_dirs.append(calculate_corr_dir(act, pred, last_value))

            pred = df[col1_lstm].iloc[i:i + hor]
            lstm_1_rmses.append(calculate_rmse(act, pred))
            lstm_1_maes.append(calculate_mae(act, pred))
            lstm_1_mapes.append(calculate_mape(act, pred))
            if last_value:
                lstm_1_corr_dirs.append(calculate_corr_dir(act, pred, last_value))

            pred = df[col2_mlp].iloc[i:i + hor]
            mlp_2_rmses.append(calculate_rmse(act, pred))
            mlp_2_maes.append(calculate_mae(act, pred))
            mlp_2_mapes.append(calculate_mape(act, pred))
            if last_value:
                mlp_2_corr_dirs.append(calculate_corr_dir(act, pred, last_value))

            pred = df[col2_lstm].iloc[i:i + hor]
            lstm_2_rmses.append(calculate_rmse(act, pred))
            lstm_2_maes.append(calculate_mae(act, pred))
            lstm_2_mapes.append(calculate_mape(act, pred))
            if last_value:
                lstm_2_corr_dirs.append(calculate_corr_dir(act, pred, last_value))

            pred = df["VAR_pred"].iloc[i:i + hor]
            var_rmses.append(calculate_rmse(act, pred))
            var_maes.append(calculate_mae(act, pred))
            var_mapes.append(calculate_mape(act, pred))
            if last_value:
                var_corr_dirs.append(calculate_corr_dir(act, pred, last_value))

            pred = df["VECM_pred"].iloc[i:i + hor]
            vecm_rmses.append(calculate_rmse(act, pred))
            vecm_maes.append(calculate_mae(act, pred))
            vecm_mapes.append(calculate_mape(act, pred))
            if last_value:
                vecm_corr_dirs.append(calculate_corr_dir(act, pred, last_value))

            pred = df["ARIMA_pred"].iloc[i:i + hor]
            arima_rmses.append(calculate_rmse(act, pred))
            arima_maes.append(calculate_mae(act, pred))
            arima_mapes.append(calculate_mape(act, pred))
            if last_value:
                arima_corr_dirs.append(calculate_corr_dir(act, pred, last_value))
                
                
            
            

        rw_rmse = np.mean(rw_rmses)

        avg_1_rmse = np.mean(avg_1_rmses)
        avg_1_mae = np.mean(avg_1_maes)
        avg_1_mape = np.mean(avg_1_mapes)
        avg_1_corr_dir = calculate_avg_dir_accuracy(avg_1_corr_dirs)
        avg_1_red = calculate_rmse_reduction(rw_rmse, avg_1_rmse)
                
        eigen_1_rmse = np.mean(eigen_1_rmses)
        eigen_1_mae = np.mean(eigen_1_maes)
        eigen_1_mape = np.mean(eigen_1_mapes)
        eigen_1_corr_dir = calculate_avg_dir_accuracy(eigen_1_corr_dirs)
        eigen_1_red = calculate_rmse_reduction(rw_rmse, eigen_1_rmse)

        avg_2_rmse = np.mean(avg_2_rmses)
        avg_2_mae = np.mean(avg_2_maes)
        avg_2_mape = np.mean(avg_2_mapes)
        avg_2_corr_dir = calculate_avg_dir_accuracy(avg_2_corr_dirs)
        avg_2_red = calculate_rmse_reduction(rw_rmse, avg_2_rmse)

        eigen_2_rmse = np.mean(eigen_2_rmses)
        eigen_2_mae = np.mean(eigen_2_maes)
        eigen_2_mape = np.mean(eigen_2_mapes)
        eigen_2_corr_dir = calculate_avg_dir_accuracy(eigen_2_corr_dirs)
        eigen_2_red = calculate_rmse_reduction(rw_rmse, eigen_2_rmse)

        avg_3_rmse = np.mean(avg_3_rmses)
        avg_3_mae = np.mean(avg_3_maes)
        avg_3_mape = np.mean(avg_3_mapes)
        avg_3_corr_dir = calculate_avg_dir_accuracy(avg_3_corr_dirs)
        avg_3_red = calculate_rmse_reduction(rw_rmse, avg_3_rmse)

        eigen_3_rmse = np.mean(eigen_3_rmses)
        eigen_3_mae = np.mean(eigen_3_maes)
        eigen_3_mape = np.mean(eigen_3_mapes)
        eigen_3_corr_dir = calculate_avg_dir_accuracy(eigen_3_corr_dirs)
        eigen_3_red = calculate_rmse_reduction(rw_rmse, eigen_3_rmse)

        avg_4_rmse = np.mean(avg_4_rmses)
        avg_4_mae = np.mean(avg_4_maes)
        avg_4_mape = np.mean(avg_4_mapes)
        avg_4_corr_dir = calculate_avg_dir_accuracy(avg_4_corr_dirs)
        avg_4_red = calculate_rmse_reduction(rw_rmse, avg_4_rmse)

        eigen_4_rmse = np.mean(eigen_4_rmses)
        eigen_4_mae = np.mean(eigen_4_maes)
        eigen_4_mape = np.mean(eigen_4_mapes)
        eigen_4_corr_dir = calculate_avg_dir_accuracy(eigen_4_corr_dirs)
        eigen_4_red = calculate_rmse_reduction(rw_rmse, eigen_4_rmse)
        
        
        
        
        
        mlp_rmse = np.mean(mlp_rmses)
        mlp_mae = np.mean(mlp_maes)
        mlp_mape = np.mean(mlp_mapes)
        mlp_corr_dir = calculate_avg_dir_accuracy(mlp_corr_dirs)
        mlp_red = calculate_rmse_reduction(rw_rmse, mlp_rmse)

        lstm_rmse = np.mean(lstm_rmses)
        lstm_mae = np.mean(lstm_maes)
        lstm_mape = np.mean(lstm_mapes)
        lstm_corr_dir = calculate_avg_dir_accuracy(lstm_corr_dirs)
        lstm_red = calculate_rmse_reduction(rw_rmse, lstm_rmse)
        
        
        
        mlp_1_rmse = np.mean(mlp_1_rmses)
        mlp_1_mae = np.mean(mlp_1_maes)
        mlp_1_mape = np.mean(mlp_1_mapes)
        mlp_1_corr_dir = calculate_avg_dir_accuracy(mlp_1_corr_dirs)
        mlp_1_red = calculate_rmse_reduction(rw_rmse, mlp_1_rmse)

        lstm_1_rmse = np.mean(lstm_1_rmses)
        lstm_1_mae = np.mean(lstm_1_maes)
        lstm_1_mape = np.mean(lstm_1_mapes)
        lstm_1_corr_dir = calculate_avg_dir_accuracy(lstm_1_corr_dirs)
        lstm_1_red = calculate_rmse_reduction(rw_rmse, lstm_1_rmse)
        
        mlp_2_rmse = np.mean(mlp_2_rmses)
        mlp_2_mae = np.mean(mlp_2_maes)
        mlp_2_mape = np.mean(mlp_2_mapes)
        mlp_2_corr_dir = calculate_avg_dir_accuracy(mlp_2_corr_dirs)
        mlp_2_red = calculate_rmse_reduction(rw_rmse, mlp_2_rmse)

        lstm_2_rmse = np.mean(lstm_2_rmses)
        lstm_2_mae = np.mean(lstm_2_maes)
        lstm_2_mape = np.mean(lstm_2_mapes)
        lstm_2_corr_dir = calculate_avg_dir_accuracy(lstm_2_corr_dirs)
        lstm_2_red = calculate_rmse_reduction(rw_rmse, lstm_2_rmse)
        
        var_rmse = np.mean(var_rmses)
        var_mae = np.mean(var_maes)
        var_mape = np.mean(var_mapes)
        var_corr_dir = calculate_avg_dir_accuracy(var_corr_dirs)
        var_red = calculate_rmse_reduction(rw_rmse, var_rmse)

        vecm_rmse = np.mean(vecm_rmses)
        vecm_mae = np.mean(vecm_maes)
        vecm_mape = np.mean(vecm_mapes)
        vecm_corr_dir = calculate_avg_dir_accuracy(vecm_corr_dirs)
        vecm_red = calculate_rmse_reduction(rw_rmse, vecm_rmse)

        arima_rmse = np.mean(arima_rmses)
        arima_mae = np.mean(arima_maes)
        arima_mape = np.mean(arima_mapes)
        arima_corr_dir = calculate_avg_dir_accuracy(arima_corr_dirs)
        arima_red = calculate_rmse_reduction(rw_rmse, arima_rmse)
        
        
        
        

        # Prepare data for CSV
        results = [
            [seg, hor, 'avg_1', avg_1_rmse, avg_1_mae, avg_1_mape, avg_1_corr_dir, avg_1_red],
            [seg, hor, 'eigen_1', eigen_1_rmse, eigen_1_mae, eigen_1_mape, eigen_1_corr_dir, eigen_1_red],
            [seg, hor, 'avg_2', avg_2_rmse, avg_2_mae, avg_2_mape, avg_2_corr_dir, avg_2_red],
            [seg, hor, 'eigen_2', eigen_2_rmse, eigen_2_mae, eigen_2_mape, eigen_2_corr_dir, eigen_2_red],
            [seg, hor, 'avg_3', avg_3_rmse, avg_3_mae, avg_3_mape, avg_3_corr_dir, avg_3_red],
            [seg, hor, 'eigen_3', eigen_3_rmse, eigen_3_mae, eigen_3_mape, eigen_3_corr_dir, eigen_3_red],
            [seg, hor, 'avg_4', avg_4_rmse, avg_4_mae, avg_4_mape, avg_4_corr_dir, avg_4_red],
            [seg, hor, 'eigen_4', eigen_4_rmse, eigen_4_mae, eigen_4_mape, eigen_4_corr_dir, eigen_4_red],
            [seg, hor, 'MLP', mlp_rmse, mlp_mae, mlp_mape, mlp_corr_dir, mlp_red],
            [seg, hor, 'LSTM', lstm_rmse, lstm_mae, lstm_mape, lstm_corr_dir, lstm_red],
            [seg, hor, 'MLP_1', mlp_1_rmse, mlp_1_mae, mlp_1_mape, mlp_1_corr_dir, mlp_1_red],
            [seg, hor, 'LSTM_1', lstm_1_rmse, lstm_1_mae, lstm_1_mape, lstm_1_corr_dir, lstm_1_red],
            [seg, hor, 'MLP_2', mlp_2_rmse, mlp_2_mae, mlp_2_mape, mlp_2_corr_dir, mlp_2_red],
            [seg, hor, 'LSTM_2', lstm_2_rmse, lstm_2_mae, lstm_2_mape, lstm_2_corr_dir, lstm_2_red],
            [seg, hor, 'VAR', var_rmse, var_mae, var_mape, var_corr_dir, var_red],
            [seg, hor, 'VECM', vecm_rmse, vecm_mae, vecm_mape, vecm_corr_dir, vecm_red],
            [seg, hor, 'ARIMA', arima_rmse, arima_mae, arima_mape, arima_corr_dir, arima_red]
    ]

        # Create a DataFrame
        columns = ['Segment', 'Horizon', 'Model', 'RMSE', 'MAE', 'MAPE', 'CorrDir', 'Reduction']
        results_df = pd.DataFrame(results, columns=columns)

        # Write to CSV
        results_df.to_csv('model_metrics.csv', index=False, header=False, mode="a")    
-1.758593271006248e-13
-4.121147867408581e-13
-1.758593271006248e-13
#df_comb1 = df.copy()

 
 


9.912071163853398e-13
9.912071163853398e-13
9.823253321883385e-13
9.823253321883385e-13


-1.758593271006248e-13